<a href="https://colab.research.google.com/github/JacePalmer/Machine-Learning/blob/main/W2_BMEG4983.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src=https://brand.uark.edu/_resources/images/UA_Logo_Horizontal.jpg width="400" height="96">

####BMEG 4983 - Genome Engineering and Synthetic Biology -
####For more information, check out the Nelson lab for Therapeutic Genome Engineering (https://nelsonlab.uark.edu/)


#2. Data Workshop 2 - Python and sequence data

Today, we are going to examine how to access information on our selected
genes, do some basic python manipulaiting strings, and introduce some data I/O.

Remember to check the table of contents button on the upper right.

Updated Jan 22, 2025

# 2.1 Outline

This workbook will cover how we can handle sequence-based data. We will start with manual access and build toward abstracting this information.


To analyze genomic data, we will need to have a way to access this data. If you have your own installation of python, you can download the genome to your own PC (Link). In google colab, we have a few options to pull in data in smaller chunks. The free version of google colab is also limited on RAM. We will keep our analysis focused on smaller sets of data.

The outline today (remember, you can view the outline using the table of contents button on the left)

*   Input and output options
 * Local - upload files from your PC
 * Google drive - you can access data in your google drive
*   Manual sequence database access (i.e. go to the website)
*   Programmatic sequence database access
 *   Data Commons
 *   Entrez via Biopython
 *   Genome Browser XML (Hacky version that can work)

A note: For all of the public database options, make sure that you access these infrequently and don't make many requests. See the rules list below.



# 2.2 Input/Output in Google colab (Local)

We can import files from our own computer if needed.

The code below will open a dialog box on your PC.

In [ ]:
#This cell allows you to import a local file. Depending on the file, you can
#decide what to do with it
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
uploaded['filename.txt']

KeyError: 'filename.txt'

# 2.3 Input/Output in Google colab (Online)

```python
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/NM_Data.csv')

```

This uses a pre-built library called Pandas. More on using pandas in a later workshop. Notice the 'raw' in the filename. Make sure you get this link, otherwise, it will try to load an HTML file. You can make your own github account or host data on your own. Github free does limit file sizes as well. You may need to do a search to figure out how to get data out of the dataframe or reorganize the dataframe. Try asking chatGPT how to do it.

Let's do an example where we pull in a small fasta file.



In [ ]:
import pandas as pd #more on pandas later, this is a good tool for importing data

#I have the mitochondrial DNA sequence stored here
file_name = 'https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/chrM.fa'

df = pd.read_csv('https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/chrM.fa')

#Notice the output is now in a 'dataframe'. It has a first line with
#a > character to indicate the name of the sequence
print(df)

#Check out top rows:
data_top = df.head()
print(data_top)

#Get column names out (we only have one)
for col in df.columns:
    header = col
    header = header.replace('>', '') #remove the marker
    print('Here is our header: '+header)
seq = ''
for index, row in df.iterrows():
    seq = seq + row['>'+header]
print('our sequence for '+header+' is '+str(len(seq))+' bp long')

                                                 >chrM
0    GATCACAGGTCTATCACCCTATTAACCACTCACGGGAGCTCTCCAT...
1    CGTCTGGGGGGTATGCACGCGATAGCATTGCGAGACGCTGGAGCCG...
2    GCAGTATCTGTCTTTGATTCCTGCCTCATCCTATTATTTATCGCAC...
3    ACAGGCGAACATACTTACTAAAGTGTGTTAATTAATTAATGCTTGT...
4    ACAATTGAATGTCTGCACAGCCACTTTCCACACAGACATCATAACA...
..                                                 ...
272  CATTTACCGTACATAGCACATTACAGTCAAATCCCTTCTCGTCCCC...
273  TCAGATAGGGGTCCCTTGACCACCATCCTCCGTGAAATCAATATCC...
274  ACTCTCCTCGCTCCGGGCCCATAACACTTGGGGGTAGCTAAAGTGA...
275  CTGGTTCCTACTTCAGGGTCATAAAGCCTAAATAGCCCACACGTTC...
276                                          ATCACGATG

[277 rows x 1 columns]
                                               >chrM
0  GATCACAGGTCTATCACCCTATTAACCACTCACGGGAGCTCTCCAT...
1  CGTCTGGGGGGTATGCACGCGATAGCATTGCGAGACGCTGGAGCCG...
2  GCAGTATCTGTCTTTGATTCCTGCCTCATCCTATTATTTATCGCAC...
3  ACAGGCGAACATACTTACTAAAGTGTGTTAATTAATTAATGCTTGT...
4  ACAATTGAATGTCTGCACAGCCACTTTCCACACAGACATCATAACA..

# 2.4 Structure of a gene and genome browser
We will do this demonstration in class together. This cell is maintained for refreshing our memory (or in the event of a snow day)

First, we need to know what a gene looks like when we go to NCBI or genome browser. Below is an image that shows some of the terms we need to know.

<img src=https://cdn.prod.website-files.com/621e95f9ac30687a56e4297e/64a8d81f6ab0d4000f523e5d_V2_1675983443470_27429827-33c6-438e-8abc-eb95ef861e80.png>

From left to right in the eukaryotic version
1. Distal control elements - these are enhancers or silencers. They can be some distance away in the 1D structure of DNA but can loop back and be in close proximity. We may discuss this in more detail later.
2. Proximal control elements - Here we are mostly talking about the promoter. This helps the cell control what cell type should be expressing this protein
3. Next which isn't really shown is the 5' untranslated region (UTR). This is made into RNA but not protein
4. Exons - These are the part of the gene that will be in the mature RNA
5. Introns - These are transcribed into RNA but removed from teh final mRNA
6. Not shown - 3' UTR, makes it to the final mRNA but is not made into protein
7. Transriptional termination - a signal that stops RNA from being made
8. Distal elements - Similar to the upstream control elements

Here is what that looks like in genome browser:

<img src=https://genome.ucsc.edu/images/multiRegionButton.png>
(Ignore the red arrow, this was the best image I could find hosted online)

A few things to note here. We are looking at the gene ACE2. This is on the X chromosome. You can see where on the chromosome we are with the little red line in p22.2. The coordinates are also displayed (chrX:15,560,138-15,602,945). So we are looking at 10s of thousands of bases at once here.

Another thing to note is that genes can run either from right to left or left to right. In this case, ACE2 is running from right to left which means the coding region will be on the reverse complment.

Genome browser also shows several isoforms here. You can see there are three versions of ACE2 that are slightly different (I can't spot the difference from this view) and one that is quite a bit shorter (the bottom one).

Introns are shown as the thin line with '<' pointing in the direction of the gene

Sequences that are part of the mature mRNA are shown as boxes. There are thicker boxes which represent coding regions and thinner boxes that represent the UTR. ACE2 has a fairly small 5' UTR (which is on the right b/c this gene is running right to left). The bottom variant has a much larger 3'UTR (running off the side on the left). The top 3 variants are running off the page here. We can move our window or zoom out to examine the remainder of the gene.

Go to genome browser at UCSC and find your gene. Treat this like google maps. Try zooming in until you see the amino acid sequence. Try zooming in until you see the DNA sequence.  Look at the beginning of exons. What two bp are right before every exon? What two bp are right at the end of every exon?



#2.5 Dealing with Strings
After you get your gene sequence into google colab, this is a good chance to tinker with strings. Try some of the following.

## 2.5.1 Counting nucleotides
Try implementing the code from workbook 1 to the sequence from mitochondria


In [ ]:
#Placeholder code for counting nucleotides
print(seq) #If you still have this variable active from 2.3
#you can also make your own sequence below


GATCACAGGTCTATCACCCTATTAACCACTCACGGGAGCTCTCCATGCATTTGGTATTTTCGTCTGGGGGGTATGCACGCGATAGCATTGCGAGACGCTGGAGCCGGAGCACCCTATGTCGCAGTATCTGTCTTTGATTCCTGCCTCATCCTATTATTTATCGCACCTACGTTCAATATTACAGGCGAACATACTTACTAAAGTGTGTTAATTAATTAATGCTTGTAGGACATAATAATAACAATTGAATGTCTGCACAGCCACTTTCCACACAGACATCATAACAAAAAATTTCCACCAAACCCCCCCTCCCCCGCTTCTGGCCACAGCACTTAAACACATCTCTGCCAAACCCCAAAAACAAAGAACCCTAACACCAGCCTAACCAGATTTCAAATTTTATCTTTTGGCGGTATGCACTTTTAACAGTCACCCCCCAACTAACACATTATTTTCCCCTCCCACTCCCATACTACTAATCTCATCAATACAACCCCCGCCCATCCTACCCAGCACACACACACCGCTGCTAACCCCATACCCCGAACCAACCAAACCCCAAAGACACCCCCCACAGTTTATGTAGCTTACCTCCTCAAAGCAATACACTGAAAATGTTTAGACGGGCTCACATCACCCCATAAACAAATAGGTTTGGTCCTAGCCTTTCTATTAGCTCTTAGTAAGATTACACATGCAAGCATCCCCGTTCCAGTGAGTTCACCCTCTAAATCACCACGATCAAAAGGAACAAGCATCAAGCACGCAGCAATGCAGCTCAAAACGCTTAGCCTAGCCACACCCCCACGGGAAACAGCAGTGATTAACCTTTAGCAATAAACGAAAGTTTAACTAAGCTATACTAACCCCAGGGTTGGTCAATTTCGTGCCAGCCACCGCGGTCACACGATTAACCCAAGTCAATAGAAGCCGGCGTAAAGAGTGTTTTAGATCACCCCCTCCCCAATAAAGCTAAAACTCACCTGAGTTGTAAAAAACT

## 2.5.2 Translating your protein sequence

In [ ]:
#Placeholder code for translating a sequence
print(seq) #If you still have this variable active from 2.3
#you can also make your own sequence below

GATCACAGGTCTATCACCCTATTAACCACTCACGGGAGCTCTCCATGCATTTGGTATTTTCGTCTGGGGGGTATGCACGCGATAGCATTGCGAGACGCTGGAGCCGGAGCACCCTATGTCGCAGTATCTGTCTTTGATTCCTGCCTCATCCTATTATTTATCGCACCTACGTTCAATATTACAGGCGAACATACTTACTAAAGTGTGTTAATTAATTAATGCTTGTAGGACATAATAATAACAATTGAATGTCTGCACAGCCACTTTCCACACAGACATCATAACAAAAAATTTCCACCAAACCCCCCCTCCCCCGCTTCTGGCCACAGCACTTAAACACATCTCTGCCAAACCCCAAAAACAAAGAACCCTAACACCAGCCTAACCAGATTTCAAATTTTATCTTTTGGCGGTATGCACTTTTAACAGTCACCCCCCAACTAACACATTATTTTCCCCTCCCACTCCCATACTACTAATCTCATCAATACAACCCCCGCCCATCCTACCCAGCACACACACACCGCTGCTAACCCCATACCCCGAACCAACCAAACCCCAAAGACACCCCCCACAGTTTATGTAGCTTACCTCCTCAAAGCAATACACTGAAAATGTTTAGACGGGCTCACATCACCCCATAAACAAATAGGTTTGGTCCTAGCCTTTCTATTAGCTCTTAGTAAGATTACACATGCAAGCATCCCCGTTCCAGTGAGTTCACCCTCTAAATCACCACGATCAAAAGGAACAAGCATCAAGCACGCAGCAATGCAGCTCAAAACGCTTAGCCTAGCCACACCCCCACGGGAAACAGCAGTGATTAACCTTTAGCAATAAACGAAAGTTTAACTAAGCTATACTAACCCCAGGGTTGGTCAATTTCGTGCCAGCCACCGCGGTCACACGATTAACCCAAGTCAATAGAAGCCGGCGTAAAGAGTGTTTTAGATCACCCCCTCCCCAATAAAGCTAAAACTCACCTGAGTTGTAAAAAACT

## 2.5.3 Predicting ORF in a gene

In [ ]:
#placeholder code for finding ORF in genes
print(seq) #If you still have this variable active from 2.3
#you can also make your own sequence below

GATCACAGGTCTATCACCCTATTAACCACTCACGGGAGCTCTCCATGCATTTGGTATTTTCGTCTGGGGGGTATGCACGCGATAGCATTGCGAGACGCTGGAGCCGGAGCACCCTATGTCGCAGTATCTGTCTTTGATTCCTGCCTCATCCTATTATTTATCGCACCTACGTTCAATATTACAGGCGAACATACTTACTAAAGTGTGTTAATTAATTAATGCTTGTAGGACATAATAATAACAATTGAATGTCTGCACAGCCACTTTCCACACAGACATCATAACAAAAAATTTCCACCAAACCCCCCCTCCCCCGCTTCTGGCCACAGCACTTAAACACATCTCTGCCAAACCCCAAAAACAAAGAACCCTAACACCAGCCTAACCAGATTTCAAATTTTATCTTTTGGCGGTATGCACTTTTAACAGTCACCCCCCAACTAACACATTATTTTCCCCTCCCACTCCCATACTACTAATCTCATCAATACAACCCCCGCCCATCCTACCCAGCACACACACACCGCTGCTAACCCCATACCCCGAACCAACCAAACCCCAAAGACACCCCCCACAGTTTATGTAGCTTACCTCCTCAAAGCAATACACTGAAAATGTTTAGACGGGCTCACATCACCCCATAAACAAATAGGTTTGGTCCTAGCCTTTCTATTAGCTCTTAGTAAGATTACACATGCAAGCATCCCCGTTCCAGTGAGTTCACCCTCTAAATCACCACGATCAAAAGGAACAAGCATCAAGCACGCAGCAATGCAGCTCAAAACGCTTAGCCTAGCCACACCCCCACGGGAAACAGCAGTGATTAACCTTTAGCAATAAACGAAAGTTTAACTAAGCTATACTAACCCCAGGGTTGGTCAATTTCGTGCCAGCCACCGCGGTCACACGATTAACCCAAGTCAATAGAAGCCGGCGTAAAGAGTGTTTTAGATCACCCCCTCCCCAATAAAGCTAAAACTCACCTGAGTTGTAAAAAACT

#2.6 Getting data from public sources




##2.6.1 The manual way

Explore genome browser: https://genome.ucsc.edu/

Go to ncbi and find your gene. Browse around and see what information is there.

We will do this together in class on Wednesday

What happens if you have a sequence and want to know where it is in the genome?

We will cover BLAT and BLAST in class together

## 2.6.2 Data commons
More extensive guide on data commons here:

https://colab.research.google.com/github/datacommonsorg/api-python/blob/master/notebooks/analyzing_genomic_data.ipynb

We will do some programmatic access in a later workshop

In [ ]:
# Install datacommons - only need to run this once
!pip install --upgrade --quiet datacommons

#We might need some of this stuff
# Import Data Commons
import datacommons as dc

# Import other required libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import requests
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 5.0 MB/s eta 0:00:00


In [ ]:
#Example getting P53 coordinates using data commons

#First we will make a dataframe to store data
df_P53 = pd.DataFrame({'gene': ['bio/hg38_TP53']})

#This function will get the chromosome that the gene is in
df_P53['chromosome'] = df_P53['gene'].map(dc.get_property_values(df_P53['gene'], 'inChromosome'))
#print(df_P53['chromosome'].iloc[0])
#This function will get the genomic coordinates
df_P53['genomicCoordinates'] = df_P53['gene'].map(dc.get_property_values(df_P53['gene'], 'genomicCoordinates'))

#This function will get the first transcript, you may want to look at others
df_P53['rnaTranscript'] = df_P53['gene'].map(dc.get_property_values(df_P53['gene'], 'hasRNATranscript'))

P53_transcript = df_P53.iloc[0]['rnaTranscript'][0]

#We need to know if this gene is on the positive or negative strand
df_P53['posstrandOrientation'] = df_P53['gene'].map(dc.get_property_values(df_P53['gene'], 'strandOrientation'))
posstrand = False
if(df_P53['posstrandOrientation'].iloc[0][0]=='StrandOrientationPositive'):
  print('Gene is on positive strand')
elif(df_P53['posstrandOrientation'].iloc[0][0]=='StrandOrientationNegative'):
  print('Gene is on negative strand')
  posstrand=False
else:
  print('error reading strand orientation')

#This function will get the coding coordinates
temp_dict = dc.get_property_values([P53_transcript], 'codingCoordinates')

for value in temp_dict[P53_transcript]:
  tempvalue = value.replace('Position','')
  templist = tempvalue.split("To")
  codingstart = templist[0]
  codingend = templist[1]
print('Coding coordinates run from '+codingstart+' to '+codingend+ ' on chromosome '+ df_P53['chromosome'].iloc[0][0])
#This function will get exon coordinates and put them in a set
temp_dict = dc.get_property_values([P53_transcript], 'exonCoordinates')

exoncords = pd.DataFrame(columns=['exonnum', 'exonstart', 'exonend'])
#A python set that includes start and stop for each exon
for i,value in enumerate(temp_dict[P53_transcript]):
  tempvalue = value.replace('Position','')
  templist = tempvalue.split("To")
  if(posstrand):
    exonvalue = i+1
  else:
    exonvalue = len(temp_dict[P53_transcript])-(i)
  exoncords.loc[len(exoncords.index)] = ['Exon '+str(exonvalue), templist[0],templist[1]]
print('exon coordinates:')
print(exoncords)

#Now we can get the sequences from the coordinates. Let's use NIH Entrez tool for that

Gene is on negative strand
Coding coordinates run from 7669608 to 7676594 on chromosome bio/hg38_chr17
exon coordinates:
    exonnum exonstart  exonend
0   Exon 11   7668401  7669690
1   Exon 10   7670608  7670715
2    Exon 9   7673534  7673608
3    Exon 8   7673700  7673837
4    Exon 7   7674180  7674290
5    Exon 6   7674858  7674971
6    Exon 5   7675052  7675236
7    Exon 4   7675993  7676272
8    Exon 3   7676381  7676403
9    Exon 2   7676520  7676622
10   Exon 1   7687376  7687538


## 2.6.3 UCSC XML

In [ ]:
#Here is a hacky version of getting sequence data by chromosome from UCSC
#The function takes in a chromosome and start and stop coords. I made it
#where it will give an error if you ask for more than 1000 bp
import pandas as pd
def getseqdatafromcoords(chr_num_as_str, start, stop):
  if(stop<start):
    print('warning, your start and stop coordinates are inverted')
    return ''
  if(stop - start > 2000):
    print('warning, you asked for a seq larger than 2000 bp')
    return ''
  url_for_seq = 'http://genome.ucsc.edu/cgi-bin/das/hg38/dna?segment=chr'+str(chr_num_as_str)+':'+str(start)+','+str(stop)
  print(url_for_seq)
  df = pd.read_csv(url_for_seq)
  seq = ''
  index = 4
  while True:
    df.iat[index,0]
    seq = seq+df.iat[index,0]
    index = index +1
    if (df.iat[index,0]=="</DNA>"):
      print('break')
      break
  return seq

get_seq = getseqdatafromcoords('X', 83803112, 83803313)
print(get_seq)

http://genome.ucsc.edu/cgi-bin/das/hg38/dna?segment=chrX:83803112,83803313
break
ttcgatttaatcattttgtgtactggtaggctttgattcctttctgtttctcctgtgtatctactgcaggatttttagtgacattgcaatgaggcttacatacagcatcctatagtgatatcagtttattttaagttgataacaacttaattttgacatgtctttattttctgttagtaccattataatacatatattgcaa


## 2.6.4 Entrez
**From Biopython guide:**

Before using Biopython to access the NCBI’s online resources (via Bio.Entrez or some of the other modules), please read the NCBI’s Entrez User Requirements. If the NCBI finds you are abusing their systems, they can and will ban your access!

**To paraphrase:**

* For any series of more than 100 requests, do this at weekends or outside USA peak times. This is up to you to obey.
* Use the https://eutils.ncbi.nlm.nih.gov address, not the standard NCBI Web address. Biopython uses this web address.
* If you are using a API key, you can make at most 10 queries per second, otherwise at most 3 queries per second. This is automatically enforced by Biopython. Include api_key="MyAPIkey" in the argument list or set it as a module level variable:
```
>>> from Bio import Entrez
>>> Entrez.api_key = "MyAPIkey"
```
* Use the optional email parameter so the NCBI can contact you if there is a problem. You can either explicitly set this as a parameter with each call to Entrez (e.g. include email="A.N.Other@example.com" in the argument list), or you can set a global email address:
```
>>> from Bio import Entrez
>>> Entrez.email = "A.N.Other@example.com"
```
* Bio.Entrez will then use this email address with each call to Entrez. The example.com address is a reserved domain name specifically for documentation (RFC 2606). Please DO NOT use a random email – it’s better not to give an email at all. The email parameter has been mandatory since June 1, 2010. In case of excessive usage, NCBI will attempt to contact a user at the e-mail address provided prior to blocking access to the E-utilities.
* If you are using Biopython within some larger software suite, use the tool parameter to specify this. You can either explicitly set the tool name as a parameter with each call to Entrez (e.g. include tool="MyLocalScript" in the argument list), or you can set a global tool name:
```
>>> from Bio import Entrez
>>> Entrez.tool = "MyLocalScript"
```
* The tool parameter will default to Biopython.
* For large queries, the NCBI also recommend using their session history feature (the WebEnv session cookie string, see Section ‍9.16). This is only slightly more complicated.

  
  |chr id for entrez | Length (bp) | Name
  |:-:|:-:|:-:
  NC_000001.11  |  248956422  |  Homo sapiens chromosome 1
  NC_000002.12  |  242193529  |  Homo sapiens chromosome 2
  NC_000003.12  |  198295559  |  Homo sapiens chromosome 3
  NC_000004.12  |  190214555  |  Homo sapiens chromosome 4
  NC_000005.10  |  181538259  |  Homo sapiens chromosome 5
  NC_000006.12  |  170805979  |  Homo sapiens chromosome 6
  NC_000007.14  |  159345973  |  Homo sapiens chromosome 7
  NC_000008.11  |  145138636  |  Homo sapiens chromosome 8
  NC_000009.12  |  138394717  |  Homo sapiens chromosome 9
  NC_000010.11  |  133797422  |  Homo sapiens chromosome 10
  NC_000011.10  |  135086622  |  Homo sapiens chromosome 11
  NC_000012.12  |  133275309  |  Homo sapiens chromosome 12
  NC_000013.11  |  114364328  |  Homo sapiens chromosome 13
  NC_000014.9   |  107043718  |  Homo sapiens chromosome 14
  NC_000015.10  |  101991189  |  Homo sapiens chromosome 15
  NC_000016.10  |  90338345   |  Homo sapiens chromosome 16
  NC_000017.11  |  83257441   |  Homo sapiens chromosome 17
  NC_000018.10  |  80373285   |  Homo sapiens chromosome 18
  NC_000019.10  |  58617616   |  Homo sapiens chromosome 19
  NC_000020.11  |  64444167   |  Homo sapiens chromosome 20
  NC_000021.9   |  46709983   |  Homo sapiens chromosome 21
  NC_000022.11  |  50818468   |  Homo sapiens chromosome 22
  NC_000023.11  |  156040895  |  Homo sapiens chromosome X
  NC_000024.10  |  57227415   |  Homo sapiens chromosome Y
  NC_012920.1   |  16569      |  Homo sapiens mitochondrion

##2.6.5 BLAST and BLAT

What if you have the opposite problem - you have a sequence and you want to know what organism it is from - or where it is located in the genome.

We can use two tools for this - BLAST (NCBI) or BLAT (UCSC).

BLAT works with genome browser to show you where your sequence is located.

Try using BLAT with your mRNA sequence to see where your sequence is located

Try using BLAST to identify what organism has the following sequence in it's genome?
```
>Unknown seqeunce
TTGAAATTTCGTAGTTTTAAATAACTTTAAGGCTTCGATAATGTTTGACA
CGTTAAGGCATGGGCATGGACATGTGAGCAGTGAATTGAGTCATGGATTC
GGGGTTCGATGGATGGGTGGTGGGGGAGTGGTGTTAATGGAGGTATTACA
CGAGGCTTACAATCAGACAGTATATGTACAAATGTCTGCCGTGGGTGCAA
CCAAAAGAGGCTTGACTCGCGAGCTGAGCAATCTATCTGCACGGCCGATA
```

##2.6.6 Programmatic access to public databases
This section is still a work in progress. Feel free to use some of the code from it as needed. We will probably skip this section in class


#2.7 Biopython and getting sequence data



##2.7.1 Extended Code
* A, T, C, G – already defined
* N = “Any” = A,T,C, or G
* R = “puRine” = G or A
* Y = “pYrimidine” = C or T
* S = “Strong” = G or C
* W = “Weak” = A or T
* K = “Keto” = G or T
* M = “Amino” = A or C
* B = “anything but A” = T, C, or G
* D = “anything but C” = A, T, or G
* H = “Anything but G” = A,T, or C
* V = “anything but T” = A, C, or G

## 2.7.2 Getting Biopython running on google colab

In [ ]:
#This code block tries to print the biopython version. If that fails, it will
#install biopython. When you run this cell again, it won't install and will
#simply check if it's installed and print the version
try:
  print("Biopython version:", Bio.__version__)
except:
  !pip install biopython #(uncomment - need to do this once per session)
  import Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.0 MB/s eta 0:00:00


## 2.7.3 Example sequence analysis

In [ ]:
import os
import sys

import urllib.request
from urllib.request import urlretrieve

import Bio
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq



from Bio.SeqUtils import gc_fraction

#Our function from last week as an example
def dna_count2(s):
  return s.count("A"), s.count("G"), s.count("C"), s.count("T")

print("Biopython version:", Bio.__version__) #comfirm it works
input_file = "unknown-sequence.fa"

fasta_loc = ("https://raw.githubusercontent.com/chris-rands/"
             "biopython-coronavirus/master/unknown-sequence.fa")

#This is one route to import data from github into biopython
if not os.path.exists(input_file):
    urlretrieve(fasta_loc, input_file)
for record in SeqIO.parse(input_file, "fasta"):
    print(record.id)

print(record.seq)

(num_a, num_g, num_c, num_T) = dna_count2(record.seq)
print("There are "+str(num_a)+" A nucleotides in the unknown seq")
#print the GC content using built in package
print('The GC percentage is: '+str(round((Bio.SeqUtils.gc_fraction(record.seq)*100),2))+'%')
print(record.seq.reverse_complement())
my_dna = Seq("ATCGnnNNatcgR")
print(my_dna.reverse_complement())

Biopython version: 1.80
Unknown_sequence
...
There are 8954 A nucleotides in the unknown seq
The GC percentage is: 37.97%
...
YcgatNNnnCGAT


#2.8 Challenge Problems
1. Write a function to reverse complement a sequence
2. Write a function to transcribe a sequence of DNA into RNA
3. Write a function to translate a sequence of DNA (or RNA, or both) into amino acids
4. Eight queens problem - write a recursive algorithm to solve the 8 queens problem?
5. More adavanced - how many solutions are there?
5. More advanced - can you solve an n queens problem? How does the amount of time scale with N?

##2.8.1 Review: Reverse Complement function
Above, we discussed the double strand nature of DNA. There are two strands that are reversed and the complement of each other:

<img src="https://cdn.kastatic.org/ka-perseus-images/8b34811a58635746dd54952b8d182fa1f0322170.png">

A common technique is to reverse and complement a sequence of DNA. It is rare to do either reverse or complement alone and they are usually done together. Let's try to write our own function. The goal of this function is to take a sequence and give us the reverse complement

To reverse commplement DNA, we need to perform two operations

1. Reverse the DNA strand
2. Complement the DNA strand (replace A with T, etc)

We showed a quick way to reverse a string above
```python
reverse_seq = my_seq[::-1]
```

Now to complement. We can't just replace them one at a time without overwriting
We need an intermediate
```python
my_seq_upper = upper(my_seq)
my_seq_rc = my_seq_upper.replace('A','t')
#Other
#Now upper case

```

In [ ]:
def reverse_complement(my_seq):
  print(my_seq)
  reverse_seq = my_seq[::-1]
  print(reverse_seq)
  my_seq_upper = reverse_seq.upper()
  print(my_seq_upper)

  my_seq_rc = my_seq_upper.replace('A','t')
  print(my_seq_rc)
  my_seq_rc = my_seq_rc.replace('T','a')
  print(my_seq_rc)
  my_seq_rc = my_seq_rc.replace('C','g')
  print(my_seq_rc)
  my_seq_rc = my_seq_rc.replace('G','c')
  print(my_seq_rc)
  my_seq_rc = my_seq_rc.upper()
  print(my_seq_rc)
  return my_seq_rc

print(reverse_complement('atcg'))

atcg
gcta
GCTA
GCTT
GCaa
Ggaa
cgaa
CGAA
CGAA


* A, T, C, G – already defined
* N = “Any” = A,T,C, or G
* R = “puRine” = G or A
* Y = “pYrimidine” = C or T
* S = “Strong” = G or C
* W = “Weak” = A or T
* K = “Keto” = G or T
* M = “Amino” = A or C
* B = “anything but A” = T, C, or G
* D = “anything but C” = A, T, or G
* H = “Anything but G” = A,T, or C
* V = “anything but T” = A, C, or G

In [ ]:
def reverse_complement_ambiguous(my_seq):
 #Make a dictionary with ambiguous symbols
  complements = { "A":"T", "T":"A","C":"G","G":"C","N":"N","R":"Y","Y":"R",
                 "S":"S","W":"W","K":"M","M":"K","B":"V","V":"B","D":"H","H":"D",
                 "a":"t", "t":"a","c":"g","g":"c","n":"n","r":"y","y":"r",
                 "s":"s","w":"w","k":"m","m":"k","b":"v","v":"b","d":"h","h":"d"}
  #This line is doing the wor. It is iterating through a reversed my_seq
  #and putting each nucleotide into the variable base. For each of those
  #it looks up in the dictionary what the complement is and adds those to
  #a growing string.
  return "".join(complements[base] for base in reversed(my_seq))

#Example
print(reverse_complement_ambiguous('ATGCTGANNNnnnRRGatcg'))

cgatCYYnnnNNNTCAGCAT


## 2.8.2 Transcribe
Replace T with U

In [ ]:
#Write a function to transcribe a sequence
def transcribe(dna_sequence):

  rna_sequence = dna_sequence.replace('T', 'U')
  return rna_sequence

# Example usage:
my_dna = "ATCGT"
my_rna = transcribe(my_dna)
print(my_rna) # Output: AUCGU

AUCGU


##2.8.3 Translate

We need to use the codon chart. I'll save you some time and make a dictionary that has all of the codons in it.


```python
codon_table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                 
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
```

In [ ]:
#Write a function to translate the following sequence of DNA:
my_seq = "GGCGAAAACGAGGAGGATATTACAATAAATGGA"

def translate(sequence):

  codon_table = {
          'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
          'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
          'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
          'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
          'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
          'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
          'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
          'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
          'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
          'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
          'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
          'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
          'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
          'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
          'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
          'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
      }
  protein_sequence = ""
  for i in range(0, len(sequence) - 2, 3):
    codon = sequence[i:i + 3]
    if codon in codon_table:
      protein_sequence += codon_table[codon]
    else:
      protein_sequence += "X"  # Unknown codon
  return protein_sequence

# Example usage
my_seq = "GGCGAAAACGAGGAGGATATTACAATAAATGGA"
protein = translate(my_seq)
print(protein)

GENEEDITING


That was genuinely clever lmao I thought I messed up

## 2.8.4 Finding ORF in a sequence
An open reading frame (ORF) is defined as a sequence that begins with a start codon 'ATG' and ends with one of the following stop codons 'TAA', 'TAG', 'TGA'

Could you write a function that takes a sequence as input (a string) and returns the longest ORF? Or a list of all ORFs?

In [ ]:
import re

def find_orfs(sequence):
    """Finds all Open Reading Frames (ORFs) in a given DNA sequence and returns the longest one."""
    start_codon = "ATG"
    stop_codons = {"TAA", "TAG", "TGA"}
    orfs = []

    for frame in range(3):
        i = frame
        while i < len(sequence) - 2:
            if sequence[i:i+3] == start_codon:
                for j in range(i+3, len(sequence) - 2, 3):
                    codon = sequence[j:j+3]
                    if codon in stop_codons:
                        orfs.append(sequence[i:j+3])
                        break
            i += 3

    return max(orfs, key=len) if orfs else None, orfs

# Example usage:
dna_sequence = "AAGATGCGTAAATGCCCTAGATGTGA"
longest_orf, all_orfs = find_orfs(dna_sequence)
print("Longest ORF:", longest_orf)
print("All ORFs:", all_orfs)

Longest ORF: ATGCCCTAG
All ORFs: ['ATGCCCTAG', 'ATGTGA']


##2.8.5 Eight queens problem intro (Recursion)

In two weeks, we will cover DNA alignment. To perform DNA alignment we need to have a little background.

First we need a program that can look deeply into many possible alignments. Recursion is a process where a function calls on itself. This allows for unique
functionality distinct from for loops and while loops

A good example of this is the 8 queens problem. Imagine you are provided a chess board and 8 queens. The goal is to place 8 queens anywhere on the board such that they cannot attack each other.

<img src="https://raw.githubusercontent.com/chrisnelsonlab/DIY-Genome-2021/main/images/8queens.png" width="300">


It may be simpler to start with a 4x4 chess board and try to place 4 queens (there are 2 solutions to this version (technically it's only one by symmetry). Try this excercise on a piece of paper or on the image below. Pay attention to your mental process as you try and solve.


<img src="https://raw.githubusercontent.com/chrisnelsonlab/DIY-Genome-2021/main/images/4queens.png" width="250">


*For help, check workbook 4 or look online for solutions.*

In [ ]:
def is_safe(board, row, col, n):
    """Check if a queen can be placed at board[row][col] without being attacked."""
    for i in range(col):
        if board[row][i]:
            return False

    for i, j in zip(range(row, -1, -1), range(col, -1, -1)):
        if board[i][j]:
            return False

    for i, j in zip(range(row, n, 1), range(col, -1, -1)):
        if board[i][j]:
            return False

    return True

def solve_n_queens_util(board, col, n, solutions):
    """Recursively solve the N-Queens problem and store solutions."""
    if col >= n:
        solutions.append([row[:] for row in board])
        return

    for i in range(n):
        if is_safe(board, i, col, n):
            board[i][col] = 1
            solve_n_queens_util(board, col + 1, n, solutions)
            board[i][col] = 0

def solve_n_queens(n):
    """Solves the N-Queens problem and returns all solutions."""
    board = [[0] * n for _ in range(n)]
    solutions = []
    solve_n_queens_util(board, 0, n, solutions)
    return solutions

# Example usage:
n = 4  # Change to 8 for the full 8-Queens problem
solutions = solve_n_queens(n)
for sol in solutions:
    for row in sol:
        print(" ".join("Q" if cell else "." for cell in row))
    print()

. . Q .
Q . . .
. . . Q
. Q . .

. Q . .
. . . Q
Q . . .
. . Q .



## FUTURE: 2.8.X Dynamic programming

One way to speed up recursive algorithms is to remember information from previous iterations. A great example of this is calculating the Fibonacci sequence.
[This will be a topic for future classes - if you have interest in knowing more about this, contact me!]

#2.9 Tutorial List Updated

First notebook example:
https://colab.research.google.com/

Using Markdown
https://colab.research.google.com/notebooks/markdown_guide.ipynb

Importing Libraries: https://colab.research.google.com/notebooks/snippets/importing_libraries.ipynb

Using Forms:
https://colab.research.google.com/notebooks/forms.ipynb

Data input/output:
https://colab.research.google.com/notebooks/io.ipynb

Data Commons:
https://colab.research.google.com/github/datacommonsorg/api-python/blob/master/notebooks/analyzing_genomic_data.ipynb

Biopython example:
https://colab.research.google.com/github/chris-rands/biopython-coronavirus/blob/master/biopython-coronavirus-notebook.ipynb

Machine learning + bioinformatics
https://notebook.community/lilleswing/deepchem/examples/tutorials/21_Introduction_to_Bioinformatics

#2.10 Learning Check



In [ ]:
#@title Learning Check (input answers and hit play button)

#@markdown Q1: I've been to UCSC genome browser and found my gene
answer1 = 'Yes' #@param [" ","Yes", "No"]

#@markdown Q2: I've been to NCBI and found my gene
answer2 = 'Yes' #@param [" ","Yes", "No"]


#@markdown Q3: What two nucleotides were before every exon in your gene?

#@markdown If your gene is running from right to left, don't forget to reverse complement
answer3 = 'AG' #@param [" ","CG", "GG","GT","NN","AG", "GA"]


#@markdown Q4: Which one of these would be first in order for a gene?
answer4 = 'promoter' #@param [" ","intron", "exon","promoter","5'UTR","3'UTR", "Transcription Stop"]


#@markdown Q5: What organism is this sequence likely from?

#@markdown Sequence = "cgcaatttacaacacagactttatgagtgtctctatagaaatagagatgttgacacagac"
answer5 = "SARS-CoV-2" #@param ["", "human", "mouse", "aligator", "SARS-CoV-2"]

if (answer1 == "Yes"):
  print('Q1: Good')
else:
  print('Q1: Go to genome browser!')

if (answer2 == "Yes"):
  print('Q2: Good')
else:
  print('Q2: Go to NCBI!')

if (answer3 == "AG"):
  print('Q3: Correct')
else:
  print('Q3: Incorrect')

if (answer4 == "promoter"):
  print('Q3: Correct')
else:
  print('Q3: Incorrect')
if (answer5 == "SARS-CoV-2"):
  print('Q4: Correct')
else:
  print('Q4: Incorrect')

Q1: Good
Q2: Good
Q3: Correct
Q3: Correct
Q4: Correct


# 2.? Sequence Databases


https://colab.research.google.com/github/datacommonsorg/api-python/blob/master/notebooks/analyzing_genomic_data.ipynb

In [ ]:
# Install datacommons
!pip install --upgrade --quiet datacommons
# Import Data Commons
import datacommons as dc

# Import other required libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import requests
import json

df_RUNX1 = pd.DataFrame({'gene': ['bio/hg38_RUNX1']})
# Grab the chromosome and genomic coordinates using get_property_values
df_RUNX1['chromosome'] = df_RUNX1['gene'].map(dc.get_property_values(df_RUNX1['gene'], 'inChromosome'))
df_RUNX1['genomicCoordinates'] = df_RUNX1['gene'].map(dc.get_property_values(df_RUNX1['gene'], 'genomicCoordinates'))

# display the genomic coordinates
print(df_RUNX1)

# define start and stop of RUNX1
start, stop = df_RUNX1['genomicCoordinates'][0][0].strip('Position').split('To')
start = int(start)
stop = int(stop)

# 2.11 HW2

Note about genes, RNA, and protein. Let's stick with this guide: https://www.jci.org/kiosk/publish/genestyle

So my gene (*TP53* the gene will be italicisized and capital).
My mRNA would be *TP53* mRNA and the protein would be p53


For the gene you chose, answer the following questions:


## 2.11.1 Q1 Provide an introduction to your gene in Markdown

>Pick a gene that is related to a human disease. Do your best to pick different genes than the rest of the class. Briefly describe the function and what is known in an introductory paragraph. Cite your references in an appropriate format. (suggestion 250 words max, 2-5 references).

### Introduction to *Gene Choice* (replace with yours)
*TP53*, also known as tumor protein p53, is a tumor suppressor gene that plays a critical role in the regulation of cell growth and division. It is located on chromosome 17 and encodes for a protein called p53, which acts as a transcription factor that controls the expression of genes involved in cell cycle arrest, DNA repair, and apoptosis (cell death) (1). Mutations in TP53 are among the most common genetic alterations found in human cancer, and the p53 protein is considered the "guardian of the genome" because of its role in preventing the development of cancer (2).

The p53 protein is activated in response to cellular stress, such as DNA damage, and functions by binding to DNA and regulating the expression of genes involved in cell cycle arrest, DNA repair, and apoptosis. When the cell's DNA is damaged and cannot be repaired, p53 activates the process of apoptosis, which leads to the death of the cell. This helps prevent the spread of potentially cancerous cells (1,3).

The TP53 gene can be mutated in a variety of ways, including point mutations, insertions, and deletions. These mutations can result in the loss of p53 function, leading to the development of cancer. In fact, it is estimated that over 50% of human cancers have mutations in TP53 (4). The most common types of cancer associated with TP53 mutations include lung cancer, breast cancer, ovarian cancer, and colon cancer (5).

In conclusion, TP53 is a crucial tumor suppressor gene thatplays a critical role in the regulation of cell growth and division by encoding for the p53 protein. The p53 protein acts as a transcription factor that controls the expression of genes involved in cell cycle arrest, DNA repair, and apoptosis. Mutations in the TP53 gene are among the most common genetic alterations found in human cancer and can result in the loss of p53 function, leading to the development of cancer. The most common types of cancer associated with TP53 mutations include lung cancer, breast cancer, ovarian cancer, and colon cancer.

It is important to note that the TP53 gene has many functions and is not only limited to cancer development, it also plays a role in aging, immune system, and other physiological functions. Studies have shown that TP53 mutations can also cause different types of genetic disorders like Li-Fraumeni syndrome, which is a rare disorder that predisposes individuals to a wide variety of cancers. In addition to that, the p53 protein also plays a key role in regulating the immune system and inflammatory response, by modulating the activity of immune cells and promoting the clearance of infected or damaged cells.

In terms of future research, more studies are needed to fully understand the mechanisms by which TP53 mutations lead to cancer, as well as to develop effective therapies that target these mutations. Targeted therapies that restore the function of the p53 protein or inhibit the activity of mutant p53 proteins are currently being developed and tested in clinical trials, and show promising results for the treatment of cancers with TP53 mutations (6,7).





**References:**

[1] Levine, A.J. (1997). p53, the cellular gatekeeper for growth and division. Cell, 88(3), 323-331. - This one is real

[2] Vousden, K.H., & Lu, X. (2013). Live or let die: the cell's response to p53. Nature Reviews Cancer, 13(5), 275-287. - This one is real

[3] Lane, D.P., & Crawford, L.V. (1979). Isolation of a new structural gene, p53, encoding a phosphoprotein associated with cell transformation. Nature, 278(5705), 261-263. - This is a weird one. I think it picked up this (https://www.nature.com/articles/278261a0)

[4] Soussi, T., & Wiman, K.G. (2007). p53: a guardian of the genome or a double-edged sword? Nature Reviews Cancer, 7(8), 523-531.- I'm pretty sure ChatGPT made this one up

[5] Lozano, G. (2012). The p53 tumour suppressor gene: 20 years of p53 research. Nature Reviews Cancer, 12(7), 553-567.- I'm pretty sure ChatGPT made this one up

[6] Dominguez-Sola, D. et al. (2019) Targeting the p53 pathway in cancer: challenges and opportunities. Nature Reviews Cancer. 19(3, pp.187-204) - I'm pretty sure ChatGPT made this one up

[7] Jia, L. et al. (2020) p53 pathway-targeted therapy: current status and future prospects. Cancer Letters. 472, pp.15-26. Also made up?

**-Written by ChatGPT, spot edited by prof Nelson**




## 2.11.2 Q2
>What kind of gene therapy or genome editing strategy do you expect could be used to treat this disease by targeting this gene? Consider DNA repair, transcriptional activation or transcriptional repression. (suggestion 250 words max)


There are several different gene therapy and genome editing strategies that could potentially be used to treat diseases by targeting TP53.

One strategy would be to use DNA repair techniques to correct mutations in the TP53 gene. This could involve using techniques such as homologous recombination or non-homologous end joining to repair the damaged DNA. Another approach would be to use transcriptional activation techniques to increase the expression of wild-type TP53 in cells with mutations. This could be achieved by using techniques such as CRISPR-mediated transcriptional activation, which uses a modified form of the CRISPR-Cas9 system to activate gene expression (1).

Another approach would be to use transcriptional repression techniques to inhibit the activity of mutant TP53 proteins. This could be achieved by using techniques such as RNA interference or CRISPR-mediated transcriptional repression (CRISPRi) (2).

Another strategy that has been proposed is to use genome editing techniques to introduce a functional copy of the TP53 gene in cells with TP53 mutations. This could be achieved by using techniques such as CRISPR-mediated gene therapy, which uses the CRISPR-Cas9 system to introduce a functional copy of the TP53 gene into cells (3).

In conclusion, there are several different gene therapy and genome editing strategies that could be used to treat diseases by targeting TP53, including DNA repair, transcriptional activation, and transcriptional repression. Each of these strategies has its own potential benefits and limitations, and further research is needed to determine which strategies will be most effective in the clinic.

**References:**

[1] Liu, Y. et al. (2018) CRISPR-mediated transcriptional activation. Nature Methods. 15(11), pp.793-796

[2] Ran, F.A. et al. (2013) Double nicking by RNA-guided CRISPR Cas9 for enhanced genome editing specificity. Cell. 154(6), pp.1380-1389

[3] Nelson, C.E. et al. (2019) Progress and challenges in CRISPR-mediated therapeutics. Nature Reviews Genetics. 20(11), pp.637-652 - *Now I know this one is made up. That's me and I didn't write that article. I'm honored that chatGPT would make up a reference for me though*

**-chatGPT**


## 2.11.3 Q3
What are the genomic coordinates of your gene? Use hg38.

The coordinates of *TRSP2* from hg38 are:

chr22:43955442-44331714

## 2.11.4 Q4

What is the **cDNA sequence** of the most common isoform. If you are in doubt about the isoform, you can use the longest isoform. You can do this with code below or using online tools.

Document here how you collected the sequence:
I chose transcript variant 1 for TRSP2 documented here:
https://www.ncbi.nlm.nih.gov/nucleotide/CP034501.1?report=genbank&log$=nuclalign&blast_rank=4&RID=URJ19R12013

This isoform is encoded by:

```
>NM_000546.6 Homo sapiens tRNA-SeC (anticodon TCA) 2-1 [TRSP2/TRU-TCA2-1], transcript variant 1, tRNA
CGAGCCTACTAGGAGTCACCAGACCCCACGTCCGAAGTTTGGACACCGACAGATCACTGTCTCACCAAGTTAAGGTGGAAACAT
```
I used the code below to find the largest open reading frame in this sequence

In [1]:
#Placeholder function for translating a sequence
codon_table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
def seq_translate(my_seq):
  amino_acid_seq = ''
  for i in range(0,len(my_seq),3):
    amino_acid_seq = amino_acid_seq + codon_table[my_seq[i:i+3]]
  return amino_acid_seq

example_AA = seq_translate('ATGTTATCGGGGTGA')
print(example_AA)

MLSG_


In [6]:
codon_table = {
    'ATA': 'I', 'ATC': 'I', 'ATT': 'I', 'ATG': 'M',
    'ACA': 'T', 'ACC': 'T', 'ACG': 'T', 'ACT': 'T',
    'AAC': 'N', 'AAT': 'N', 'AAA': 'K', 'AAG': 'K',
    'AGC': 'S', 'AGT': 'S', 'AGA': 'R', 'AGG': 'R',
    'CTA': 'L', 'CTC': 'L', 'CTG': 'L', 'CTT': 'L',
    'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCT': 'P',
    'CAC': 'H', 'CAT': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGT': 'R',
    'GTA': 'V', 'GTC': 'V', 'GTG': 'V', 'GTT': 'V',
    'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCT': 'A',
    'GAC': 'D', 'GAT': 'D', 'GAA': 'E', 'GAG': 'E',
    'GGA': 'G', 'GGC': 'G', 'GGG': 'G', 'GGT': 'G',
    'TCA': 'S', 'TCC': 'S', 'TCG': 'S', 'TCT': 'S',
    'TTC': 'F', 'TTT': 'F', 'TTA': 'L', 'TTG': 'L',
    'TAC': 'Y', 'TAT': 'Y', 'TAA': '_', 'TAG': '_',
    'TGC': 'C', 'TGT': 'C', 'TGA': '_', 'TGG': 'W'
}

def seq_translate(my_seq):
    """Translates a DNA sequence into an amino acid sequence, handling invalid codons."""
    amino_acid_seq = ''
    for i in range(0, len(my_seq) - (len(my_seq) % 3), 3):  # Ensure full codons
        codon = my_seq[i:i+3]
        amino_acid_seq += codon_table.get(codon, "X")  # Use .get() to handle invalid codons safely
    return amino_acid_seq

# The provided TRSP2 transcript sequence
my_transcript = 'CGAGCCTACTAGGAGTCACCAGACCCCACGTCCGAAGTTTGGACACCGACAGATCACTGTCTCACCAAGTTAAGGTGGAAACAT'
print(f"Original DNA sequence: {my_transcript}")

# Translate the sequence
translated_AA = seq_translate(my_transcript)
print(f"Translated Amino Acid Sequence: {translated_AA}")

# Find the longest ORF
longest_orf = 0
longest_start = -1
longest_end = -1

for i in range(len(my_transcript) - 2):
    if my_transcript[i:i + 3] == 'ATG':  # Look for start codon
        for j in range(i + 3, len(my_transcript) - 2, 3):
            if my_transcript[j:j + 3] in ('TAG', 'TAA', 'TGA'):  # Stop codon found
                orf_length = j - i
                if orf_length > longest_orf:
                    longest_orf = orf_length
                    longest_start = i
                    longest_end = j + 3  # Include stop codon
                break  # Stop looking for stop codons after the first valid one

if longest_start != -1 and longest_end != -1:
    longest_orf_seq = my_transcript[longest_start:longest_end]
    print(f'The longest ORF has a size of: {longest_orf} bp, start: {longest_start}, stop: {longest_end}')
    print(f'The amino acid sequence of the longest ORF is: {seq_translate(longest_orf_seq)}')
else:
    print("No valid ORF found in the sequence.")

Original DNA sequence: CGAGCCTACTAGGAGTCACCAGACCCCACGTCCGAAGTTTGGACACCGACAGATCACTGTCTCACCAAGTTAAGGTGGAAACAT
Translated Amino Acid Sequence: RAY_ESPDPTSEVWTPTDHCLTKLRWKH
No valid ORF found in the sequence.


##2.11.5 Q5
What is the **amino acid sequence** of the sequence from Q4?

Document here how you arived at the sequence:
To find the amino acid sequence of the sequence from Q4, I followed these steps:

Identify the Longest Open Reading Frame (ORF)

Scanned the provided DNA sequence for the start codon (ATG).
From each start codon, checked for the first stop codon (TAA, TAG, TGA).
Measured the length of the ORFs and selected the longest one.
Extract the Longest ORF Sequence

Retrieved the nucleotide sequence corresponding to the longest ORF.
Translate the ORF into an Amino Acid Sequence

Used the standard codon table to convert each triplet (codon) into its respective amino acid.
I found the longest ORF as documented in the code above.

```
>TRSP2/1 amino acid sequence
It appears that no valid open reading frame (ORF) was found within the given sequence. This means that there is either no start codon (ATG) present or no appropriate stop codon (TAA, TAG, TGA) that follows in a proper reading frame.
```


In [ ]:
#placeholder code if needed